### This notebook is used for capstone project for IBM Data Science Professional certificate

In [31]:
import pandas as pd  
import numpy as np
import requests 
from bs4 import BeautifulSoup
import pprint
import folium

Make a dictionary of all Toronto Boroughs and Neighborhoods with Postal Codes

In [24]:

#neighborhoods_dict = {'M5G': [['Downtown Toronto'],['Central', 'Bay Street']]}

neighborhoods_dict = {}

#I can't control wikipedia, so I downloaded the page and put in my github repo

with open('List of postal codes of Canada_ M - Wikipedia.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
    
match = soup.find('table', class_='wikitable sortable jquery-tablesorter')
match = match.find('tbody')

table_rows = match.find_all('tr')


#process the table founded in the wiki page and make a dictionary of following form:
#{'Code': [['Borough'],['NeighborhoodS',...']]}
for item in table_rows:
    table_data = item.find_all('td')
        
    for i, data in enumerate(table_data):
        table_data[i] = data.text.rstrip()
  
    #process the table_data to dict: {'Code': [['Borough'],['NeighborhoodS',...']]}
    code = table_data[0]
    borough = table_data[1]
    neighborhood = table_data[2]
    
    if borough == 'Not assigned':  #filter all entries where borough is undefined
        pass
    elif code not in neighborhoods_dict: 
        if neighborhood != 'Not assigned':
            neighborhoods_dict[code]=[[borough],[neighborhood]]
        else:
            neighborhoods_dict[code]=[[borough],[borough]]
    elif code in neighborhoods_dict:
            neighborhoods_dict[code][1].append(neighborhood)
    else:
        print('something wrong with data') #check if something inconsistent in data
   

#### Let's transform our dictionary to panda's dataframe


In [25]:
# define the dataframe columns
column_names = ['PostalCode','Borough','Neighborhood']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns = column_names)

#Take a look at the empty dataframe to confirm that the columns are as intended.
neighborhoods


,PostalCode,Borough,Neighborhood


In [26]:
for k, v in neighborhoods_dict.items():
 
    borough = v[0][0]
    Neighborhood = v[1]
  
    neighborhoods = neighborhoods.append({'PostalCode': k, 'Borough': borough, 'Neighborhood': Neighborhood} , ignore_index=True)

neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Harbourfront, Regent Park]"
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
4,M7A,Queen's Park,[Queen's Park]


In [27]:
print(neighborhoods.shape)

(103, 3)


In [28]:
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
neighborhoods = neighborhoods.set_index('PostalCode').join(coord.set_index('Postal Code'))
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,[Parkwoods],43.753259,-79.329656
M4A,North York,[Victoria Village],43.725882,-79.315572
M5A,Downtown Toronto,"[Harbourfront, Regent Park]",43.654260,-79.360636
M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763
M7A,Queen's Park,[Queen's Park],43.662301,-79.389494


In [30]:
neighborhoods = neighborhoods.reset_index(level=None)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Harbourfront, Regent Park]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763
4,M7A,Queen's Park,[Queen's Park],43.662301,-79.389494
